# Avance 2

In [12]:
from src.database.db_facade import DatabaseFacade
from src.database.query_builder import SQLQueryBuilder
from sqlalchemy.exc import ResourceClosedError


In [3]:
# Fachada para abstraer las funciones de Database Connection y Database Querier
facade = DatabaseFacade()


In [4]:
# Query con Builder
builder = (
    SQLQueryBuilder()
    .select("ProductName")
    .from_table("products")
    .where("categoryID = 1 OR 1=1 --")
    .limit(10)
)
df = facade.execute_query(builder)



+-----------------------------------+
|             RESULTADOS            |
+-----------------------------------+
|                       ProductName |
|               Flour - Whole Wheat |
|        Cookie Chocolate Chip With |
|                Onions - Cippolini |
|        Sauce - Gravy; Au Jus; Mix |
|            Artichokes - Jerusalem |
|        Wine - Magnotta - Cab Sauv |
|        Table Cloth - 53x69 Colour |
|                  Halibut - Steaks |
|                    Rabbit - Whole |
|                       Scampi Tail |
|              Garbage Bags - Clear |
|              Ezy Change Mophandle |
|                        Water; Tap |
|                Beef - Top Sirloin |
|             Spoon - Soup; Plastic |
|          Kellogs Special K Cereal |
|          Wine - White; Mosel Gold |
|               Lamb - Whole; Fresh |
|                   Tea - Earl Grey |
|              Chocolate - Feathers |
|                              Kiwi |
|             Rice - Jasmine Sented |
|          

In [5]:
# Query SQL manual
query = "SELECT * FROM sales WHERE TotalPrice > 100 LIMIT 100"
df2 = facade.execute_query(query)


+---------------------------------------------------------------------------------------------------------------------------+
|                                                         RESULTADOS                                                        |
+---------------------------------------------------------------------------------------------------------------------------+
|  SalesID  SalesPersonID  CustomerID  ProductID  Quantity  Discount  TotalPrice           SalesDate      TransactionNumber |
|      416             16       71212        351        19       0.0       304.0 2025-06-05 19:51:02 QFA3F3LANPC6UVQBPYL2\r |
|      486             23       93253        417        24       0.0       552.0 2025-06-05 01:19:02 6H8SDUI9MCA1IXZ4UCUE\r |
|      659             13       65151        147        17       0.2       221.0 2025-06-06 08:49:32 1O062RUFV0LVAQ94G38J\r |
|      834              7       83198         83        22       0.0       154.0 2025-06-07 10:33:44 PY0MY0NM75BOZZ2D

In [6]:
!pytest

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.4.0, pluggy-1.6.0
rootdir: c:\Users\Pedro\Desktop\SoyHenry\Proyecto-Integrador
collected 2 items

tests\test_models.py ..                                                  [100%]

============================== 2 passed in 0.03s ==============================


# Avance 3

-2 consultas complejas:
Ventas por categoria analisis porcentual
Productos mas vendidos por pais

-Creacion de StoreProcedure Employee Metrics:
Muestra la performance de cada empleados comparandolo con el promedio de ventas.

-Creac

In [7]:
query = """WITH category_sales AS (
    SELECT
        c.CategoryID,
        c.CategoryName,
        SUM(s.TotalPrice) AS TotalSales
    FROM sales s
    JOIN products p ON s.ProductID = p.ProductID
    JOIN categories c ON p.CategoryID = c.CategoryID
    GROUP BY c.CategoryID, c.CategoryName
)
SELECT
    CategoryID,
    CategoryName,
    TotalSales,
    ROUND(100.0 * TotalSales / SUM(TotalSales) OVER (), 2) AS SalesPercentage,
    ROUND(SUM(TotalSales) OVER (ORDER BY TotalSales DESC) * 100.0 / SUM(TotalSales) OVER (), 2) AS CumulativeSalesPercentage
FROM category_sales
ORDER BY TotalSales DESC;

"""
df = facade.execute_query(query)


+----------------------------------------------------------------------------------+
|                                    RESULTADOS                                    |
+----------------------------------------------------------------------------------+
|  CategoryID CategoryName  TotalSales  SalesPercentage  CumulativeSalesPercentage |
|           1  Confections    998526.0            12.70                      12.70 |
|           7         Meat    862283.0            10.97                      23.66 |
|           9      Poultry    815645.0            10.37                      34.04 |
|           3      Cereals    805381.0            10.24                      44.28 |
|          11      Produce    736564.0             9.37                      53.64 |
|           5    Beverages    673480.0             8.56                      62.21 |
|          10       Snails    644595.0             8.20                      70.41 |
|           6      Seafood    642391.0             8.17         

Esta consulta agrupa las ventas por categoría y muestra cómo se distribuyen las ventas totales en cada una, incluyendo su porcentaje sobre el total y su porcentaje acumulado.


In [8]:
query= """
WITH sales_with_country AS (
    SELECT
        s.SalesID,
        s.TotalPrice,
        s.ProductID,
        ctry.CountryID,
        ctry.CountryName
    FROM sales s
    JOIN customers cust ON s.CustomerID = cust.CustomerID
    JOIN cities city ON cust.CityID = city.CityID
    JOIN countries ctry ON city.CountryID = ctry.CountryID
),
top_countries AS (
    SELECT
        CountryID,
        CountryName,
        SUM(TotalPrice) AS CountrySales
    FROM sales_with_country
    GROUP BY CountryID, CountryName
    ORDER BY CountrySales DESC
    LIMIT 5
),
product_sales_by_country AS (
    SELECT
        swc.CountryID,
        swc.CountryName,
        p.ProductID,
        p.ProductName,
        SUM(swc.TotalPrice) AS ProductSales
    FROM sales_with_country swc
    JOIN top_countries tc ON swc.CountryID = tc.CountryID
    JOIN products p ON swc.ProductID = p.ProductID
    GROUP BY swc.CountryID, swc.CountryName, p.ProductID, p.ProductName
),
ranked_products AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY CountryID ORDER BY ProductSales DESC) AS product_rank
    FROM product_sales_by_country
)
SELECT
    CountryName as Country,
    ProductName as Product,
    ProductSales as Sales
FROM ranked_products
WHERE product_rank <= 3
ORDER BY CountryName, ProductSales DESC;

"""
df = facade.execute_query(query)



+-----------------------------------------------------+
|                      RESULTADOS                     |
+-----------------------------------------------------+
|       Country                       Product   Sales |
| United States     Coconut - Shredded; Sweet 23871.0 |
| United States Soup - Campbells; Beef Barley 23726.0 |
| United States      Garlic - Primerba; Paste 23566.0 |
+-----------------------------------------------------+



Esta consulta trae los 3 productos más vendidos por país, ordenados por TotalPrice. Peero, como los datos provienen únicamente de EE. UU, nos aparece solo ese pais en la tabla.

# Creacion de Store Procedure

In [9]:
store_procedure_definition = """
CREATE PROCEDURE IF NOT EXISTS EmployeeMetrics()
BEGIN
    -- CTE para calcular el total de ventas por empleado
    WITH EmployeeSales AS (
        SELECT 
            SalesPersonID,
            SUM(TotalPrice) AS TotalSales
        FROM sales
        GROUP BY SalesPersonID
    ),
    -- CTE para calcular el promedio general
    SalesAvg AS (
        SELECT ROUND(AVG(TotalSales),2) AS AvgSales
        FROM EmployeeSales
    )

    -- Selección final uniendo datos de empleados con sus ventas y el promedio
    SELECT 
        e.EmployeeID,
        CONCAT(e.FirstName, ' ', e.LastName) AS EmployeeName,
        es.TotalSales,
        sa.AvgSales,
        CASE 
            WHEN es.TotalSales > sa.AvgSales THEN 'Above Average'
            WHEN es.TotalSales < sa.AvgSales THEN 'Below Average'
            ELSE 'Average'
        END AS Performance
    FROM EmployeeSales es
    JOIN employees e ON es.SalesPersonID = e.EmployeeID
    CROSS JOIN SalesAvg sa
    ORDER BY es.TotalSales DESC;
END;
"""

store_procedure_instantiation = "call EmployeeMetrics();"


In [14]:
#Bloque Try-Except para prevenir los errores de ResourceClosedError de SQL Alchemy
try:
    facade.execute_query(store_procedure_definition)
except ResourceClosedError:
    pass


In [ ]:
facade.execute_query(store_procedure_instantiation)


+------------------------------------------------------------------+
|                            RESULTADOS                            |
+------------------------------------------------------------------+
|  EmployeeID     EmployeeName  TotalSales  AvgSales   Performance |
|          23    Janet Flowers    665091.0 341896.48 Above Average |
|          22  Tonia Mc Millan    631158.0 341896.48 Above Average |
|          21     Devon Brewer    618744.0 341896.48 Above Average |
|          20    Shelby Riddle    569220.0 341896.48 Above Average |
|          19    Bernard Moody    538954.0 341896.48 Above Average |
|          18  Warren Bartlett    512154.0 341896.48 Above Average |
|          17      Seth Franco    463658.0 341896.48 Above Average |
|          16  Chadwick Walton    458800.0 341896.48 Above Average |
|          15      Kari Finley    416940.0 341896.48 Above Average |
|          14    Wendi Buckley    406812.0 341896.48 Above Average |
|          13     Katina Marks   

,EmployeeID,EmployeeName,TotalSales,AvgSales,Performance
0,23,Janet Flowers,665091.0,341896.48,Above Average
1,22,Tonia Mc Millan,631158.0,341896.48,Above Average
2,21,Devon Brewer,618744.0,341896.48,Above Average
3,20,Shelby Riddle,569220.0,341896.48,Above Average
4,19,Bernard Moody,538954.0,341896.48,Above Average
5,18,Warren Bartlett,512154.0,341896.48,Above Average
6,17,Seth Franco,463658.0,341896.48,Above Average
7,16,Chadwick Walton,458800.0,341896.48,Above Average
8,15,Kari Finley,416940.0,341896.48,Above Average
9,14,Wendi Buckley,406812.0,341896.48,Above Average


Este procedimiento primero calcula las ventas totales por empleado, luego obtiene el promedio de ventas, y finalmente compara cada empleado con ese promedio.

## Creacion de Trigger


In [17]:
# Creacion tabla products_modifications
create_product_modif_table = """
CREATE TABLE IF NOT EXISTS products_modifications (
    ModificationID INT AUTO_INCREMENT PRIMARY KEY,
    ProductID INT NOT NULL,
    UsuarioDB VARCHAR(20) NOT NULL,
    ChangeType ENUM('Insert', 'Update') NOT NULL,
    ModificationTime TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
try:
    facade.execute_query(create_product_modif_table)
except ResourceClosedError:
    pass

In [24]:
# Creacion de triggers al updatear o insertar en la tabla productos
create_insert_trigger = """
CREATE TRIGGER IF NOT EXISTS  products_modifications_insert
AFTER INSERT ON products
FOR EACH ROW
BEGIN
    INSERT INTO products_modifications (ProductID, UsuarioDB, ChangeType)
    VALUES (NEW.ProductID, CURRENT_USER(), 'Insert');
END;
"""

create_update_trigger = """
CREATE TRIGGER IF NOT EXISTS products_modifications_update
AFTER UPDATE ON products
FOR EACH ROW
BEGIN
    INSERT INTO products_modifications (ProductID, UsuarioDB, ChangeType)
    VALUES (NEW.ProductID, CURRENT_USER(), 'Update');
END; 
"""


In [25]:
try:
    facade.execute_query(create_insert_trigger)
    facade.execute_query(create_update_trigger)
except ResourceClosedError:
    pass



In [ ]:
update_query = """"
UPDATE products
SET Price = 16
WHERE ProductID=1;
"""
